In [1]:
import os
import numpy as np

In [2]:
from tars.tars_training import train_model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [4]:
data_dir = "./data/"
input_shape = 256
batch_size = 32
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = 360
input_shape = 256 
use_parallel = True
use_gpu = True
epochs = 20

In [5]:
data_transforms = {
        'train': transforms.Compose([
        transforms.Resize(scale),
        transforms.RandomResizedCrop(input_shape),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=90),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
        transforms.Resize(scale),
        transforms.CenterCrop(input_shape),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)])}

In [6]:
model_conv = torchvision.models.resnet34(pretrained = 'True')
ct = 0

for child in model_conv.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

In [7]:
#model_conv = torchvision.models.resnet34(pretrained = 'True')

In [8]:
n_class = 13

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(2048, n_class)

In [9]:
use_parallel = True
if use_parallel:
    print("[Using all the available GPUs]")
    model_conv = nn.DataParallel(model_conv, device_ids=[0])

[Using all the available GPUs]


In [10]:
data_transforms

{'train': Compose(
     Resize(size=360, interpolation=PIL.Image.BILINEAR)
     RandomResizedCrop(size=(256, 256), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
     RandomHorizontalFlip(p=0.5)
     RandomVerticalFlip(p=0.5)
     RandomRotation(degrees=(-90, 90), resample=False, expand=False)
     ToTensor()
     Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
 ), 'val': Compose(
     Resize(size=360, interpolation=PIL.Image.BILINEAR)
     CenterCrop(size=(256, 256))
     ToTensor()
     Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
 )}

In [11]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                      data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                         shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [12]:
# if use_parallel:
#     print("[Using all the available GPUs]")
#     model_conv = nn.DataParallel(model_conv, device_ids=[0, 1])

In [13]:
print("[Using CrossEntropyLoss...]")
criterion = nn.CrossEntropyLoss()

[Using CrossEntropyLoss...]


In [14]:
print("[Using small learning rate with momentum...]")
optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)

[Using small learning rate with momentum...]


In [26]:
print("[Creating Learning rate scheduler...]")
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=10, gamma=0.1)

[Creating Learning rate scheduler...]


In [27]:
model_conv

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [ ]:
print("[Training the model begun ....]")
# train_model function is here: https://github.com/Prakashvanapalli/pytorch_classifiers/blob/master/tars/tars_training.py
model_ft = train_model(model_conv, dataloaders, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, use_gpu,
                     num_epochs=epochs)

  0%|          | 0/183 [00:00<?, ?it/s]

[Training the model begun ....]
MIXUP
Epoch 0/19
----------


/home/embsysa/Desktop/pytorch_classifiers-master/tars/tars_training.py:76: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  running_loss += loss.data[0]
  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 0.0358 Acc: 0.0000


  0%|          | 0/183 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

train Loss: 0.0337 Acc: 0.0000


  0%|          | 0/183 [00:00<?, ?it/s]

val Loss: 0.0669 Acc: 0.0000

Epoch 4/19
----------


 60%|█████▉    | 109/183 [00:11<00:08,  8.98it/s]

In [21]:
#model_ft = torch.load('model_ft.pkl')

In [22]:
#torch.save(model_ft, 'model_ft.pkl')

#### Model Evaluation 

In [23]:
from tqdm import tqdm

In [24]:
correct = 0
total = 0
for data in tqdm(dataloaders['val']):
    img,label = data
    temp = model_ft(img)
    predictions = torch.argmax(temp, dim=1)
    correct += np.sum([each == predictions[pos].cpu() for pos,each in enumerate(label)])
    total += len(predictions)

100%|██████████| 79/79 [00:04<00:00, 17.88it/s]


In [25]:
correct/total

0.4305057745917961

In [ ]:
predictions

In [ ]:
label

In [ ]:
a = predictions.cpu().numpy()

np.append(a,a)

In [ ]:
# step_size = 7  --> 0.0428(19th) freeze 6 layers from back
# step_size = 10 --> 0.0399(16th) freeze 6 layers from back
# step_size = 100 --> 0.0300(20th) freeze 6 layers from back
# step_size = 1000 --> 